In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from src.patch_extractor import PatchExtractorLayer
from src.feature_extractor import FeatureExtractionLayer
from src.agregator import AggregatorLayer

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
)
training_set=train_datagen.flow_from_directory(
    r"D:\programming\models\dataset\dataset\small_train",
    batch_size=32,
    target_size=(512,512),
    class_mode='binary'
)

Found 10 images belonging to 2 classes.


In [6]:
train_batch, *_ = next(training_set)
print(type(train_batch))
print(train_batch.shape)

# Convert to TensorFlow tensor
import tensorflow as tf
train_batch = tf.convert_to_tensor(train_batch, dtype=tf.float32)
print(type(train_batch), train_batch.shape)

<class 'numpy.ndarray'>
(10, 512, 512, 3)
<class 'tensorflow.python.framework.ops.EagerTensor'> (10, 512, 512, 3)


In [7]:
test_datagen=ImageDataGenerator(
    rescale=1./255
)
test_set=test_datagen.flow_from_directory(
    r"D:\programming\models\dataset\dataset\small_test",
    
    target_size=(512,512),
    batch_size=32,
    class_mode='binary'
)

Found 10 images belonging to 2 classes.


In [8]:
test_batch, *_ = next(test_set)
print(type(test_batch))
print(test_batch.shape)

# Convert to TensorFlow tensor
import tensorflow as tf
test_batch = tf.convert_to_tensor(test_batch, dtype=tf.float32)
print(type(test_batch), test_batch.shape)

<class 'numpy.ndarray'>
(10, 512, 512, 3)
<class 'tensorflow.python.framework.ops.EagerTensor'> (10, 512, 512, 3)


In [9]:
patch_layer = PatchExtractorLayer()
feature_layer = FeatureExtractionLayer()
agg_layer = AggregatorLayer()


In [10]:
patched_training_set = patch_layer(train_batch)
patched_test_set = patch_layer(test_batch)
print("Patched training set shape:", patched_training_set)


Patched training set shape: tf.Tensor(
[[[[[0.09019608 0.06666667 0.06666667]
    [0.09019608 0.06666667 0.06666667]
    [0.08627451 0.0627451  0.0627451 ]
    ...
    [0.11764707 0.08627451 0.07843138]
    [0.12156864 0.09019608 0.08235294]
    [0.12156864 0.09019608 0.08235294]]

   [[0.09019608 0.06666667 0.06666667]
    [0.09019608 0.06666667 0.06666667]
    [0.08627451 0.0627451  0.0627451 ]
    ...
    [0.11764707 0.08627451 0.07843138]
    [0.12156864 0.09019608 0.08235294]
    [0.12156864 0.09019608 0.08235294]]

   [[0.09019608 0.06666667 0.06666667]
    [0.09019608 0.06666667 0.06666667]
    [0.08627451 0.0627451  0.0627451 ]
    ...
    [0.11764707 0.08627451 0.07843138]
    [0.12156864 0.09019608 0.08235294]
    [0.12156864 0.09019608 0.08235294]]

   ...

   [[0.14901961 0.10980393 0.07450981]
    [0.14901961 0.10980393 0.07450981]
    [0.14901961 0.10980393 0.07450981]
    ...
    [0.7058824  0.59607846 0.5137255 ]
    [0.70980394 0.6117647  0.5254902 ]
    [0.70980394 0.

In [11]:

features_training_set = feature_layer(patched_training_set)
features_test_set = feature_layer(patched_test_set)
aggregated_training_set = agg_layer(features_training_set)
aggregated_test_set = agg_layer(features_test_set)
print("Training set shape:", aggregated_training_set.shape)
print("Test set shape:", aggregated_test_set.shape)

Training set shape: (10, 24576)
Test set shape: (10, 24576)


In [12]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=512,activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=256,activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

In [13]:
model.compile(loss="binary_crossentropy",metrics=['accuracy'],optimizer="Adam")

In [ ]:
model.fit(features_training_set)